In [63]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import validation_curve, learning_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from sklearn.feature_selection import SelectKBest, f_regression




In [209]:
data = pd.read_excel("C:\\Users\\rufus\\OneDrive - The University of Manchester\\PDRA\\Sequencing\\stoke_imapact_data\\Manchester_WithMeans_250612.xlsx")
ella_data = pd.read_excel("C:\\Users\\rufus\\OneDrive - The University of Manchester\\PDRA\\Sequencing\\stoke_imapact_data\\250401_MCR_Ella.xlsx")

In [210]:
ella_data = ella_data.replace('ND',np.nan)
ella_data['IL18_V3'].astype('float64')
ella_data['PDGFBB_V4'].astype('float64')

C:\Users\rufus\AppData\Local\Temp\ipykernel_14284\4183169611.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ella_data = ella_data.replace('ND',np.nan)


0      505.0
1      584.0
2      370.0
3      528.0
4        NaN
       ...  
185      NaN
186      NaN
187      NaN
188      NaN
189      NaN
Name: PDGFBB_V4, Length: 190, dtype: float64

In [211]:
data = pd.merge(data, ella_data, on='Study_ID', how='inner')
data = data.drop(columns=['Study_ID'])

In [ ]:
#data = data.drop(range(190, 196))

In [212]:
col_to_remove = ['Date_of_Enrollment',
'Time_of_symptom_onset 🔒',
'Time_symptoms_were_recognized 🔒',
'Time_last_known_well_(LKW) 🔒',
'Date/Time_Blood_Drawn_',
'blooddrawdiff',
'MoCA_V2',
'MoCA_V3',
'MoCA_V4',
'MoCA_V5',
'6mo_imaging',
'30mo_imaging',
'Qualifying_Stroke_Date',
'V3_Date',
'V4_Date',
'V5_Date',
'V3_Post_Stroke_Days',
'V4_Post_Stroke_Days',
'V5_Post_Stroke_Days',
'TimePoint.x',
'I_feel_fatigued.x',
'I_feel_weak_all_over.x',
"I_feel_listless_('washed_out').x",
'I_feel_tired.x',
'I_have_trouble_starting_things_because_I_am_tired.x',
'I_have_trouble_finishing_things_because_I_am_tired.x',
'I_have_energy.x',
'I_am_able_to_do_my_usual_activities.x',
'I_need_to_sleep_during_the_day.x',
'I_am_too_tired_to_eat.x',
'I_need_help_doing_my_usual_activities.x',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.x',
'I_have_to_limit_my_social_activity_because_I_am_tired.x',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.x',
'Are_you_basically_satisfied_with_your_life?.x',
'Have_you_dropped_many_of_your_activities_and_interests?.x',
'Do_you_feel_that_your_life_is_empty?.x',
'Do_you_often_get_bored?.x',
'Are_you_in_good_spirits_most_of_the_time?.x',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.x',
'Do_you_feel_happy_most_of_the_time?.x',
'Do_you_often_feel_helpless?.x',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.x',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.x',
'Do_you_think_it_is_wonderful_to_be_alive_now?.x',
'Do_you_feel_worthless_the_way_you_are_now?.x',
'Do_you_feel_full_of_energy?.x',
'Do_you_feel_that_your_situation_is_hopeless?.x',
'Do_you_think_that_most_people_are_better_off_than_you_are?.x',
'TimePoint.y',
'I_feel_fatigued.y',
'I_feel_weak_all_over.y',
"I_feel_listless_('washed_out').y",
'I_feel_tired.y',
'I_have_trouble_starting_things_because_I_am_tired.y',
'I_have_trouble_finishing_things_because_I_am_tired.y',
'I_have_energy.y',
'I_am_able_to_do_my_usual_activities.y',
'I_need_to_sleep_during_the_day.y',
'I_am_too_tired_to_eat.y',
'I_need_help_doing_my_usual_activities.y',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.y',
'I_have_to_limit_my_social_activity_because_I_am_tired.y',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.y',
'Are_you_basically_satisfied_with_your_life?.y',
'Have_you_dropped_many_of_your_activities_and_interests?.y',
'Do_you_feel_that_your_life_is_empty?.y',
'Do_you_often_get_bored?.y',
'Are_you_in_good_spirits_most_of_the_time?.y',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.y',
'Do_you_feel_happy_most_of_the_time?.y',
'Do_you_often_feel_helpless?.y',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.y',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.y',
'Do_you_think_it_is_wonderful_to_be_alive_now?.y',
'Do_you_feel_worthless_the_way_you_are_now?.y',
'Do_you_feel_full_of_energy?.y',
'Do_you_feel_that_your_situation_is_hopeless?.y',
'Do_you_think_that_most_people_are_better_off_than_you_are?.y',
'TimePoint',
'I_feel_fatigued',
'I_feel_weak_all_over',
"I_feel_listless_('washed_out')",
'I_feel_tired',
'I_have_trouble_starting_things_because_I_am_tired',
'I_have_trouble_finishing_things_because_I_am_tired',
'I_have_energy',
'I_am_able_to_do_my_usual_activities',
'I_need_to_sleep_during_the_day',
'I_am_too_tired_to_eat',
'I_need_help_doing_my_usual_activities',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do',
'I_have_to_limit_my_social_activity_because_I_am_tired',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_',
'Are_you_basically_satisfied_with_your_life?',
'Have_you_dropped_many_of_your_activities_and_interests?',
'Do_you_feel_that_your_life_is_empty?',
'Do_you_often_get_bored?',
'Are_you_in_good_spirits_most_of_the_time?',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?',
'Do_you_feel_happy_most_of_the_time?',
'Do_you_often_feel_helpless?',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?',
'Do_you_think_it_is_wonderful_to_be_alive_now?',
'Do_you_feel_worthless_the_way_you_are_now?',
'Do_you_feel_full_of_energy?',
'Do_you_feel_that_your_situation_is_hopeless?',
'Do_you_think_that_most_people_are_better_off_than_you_are?',
'Is_ECG_available? 🔒',
'Are_labs_available? 🔒',
'LangFact_V3',
'WMF_V3',
'ProcSpeed_V3',
'MemFact_V3',
'Visuospatial_V3',
'GlobalTest_V3',
'GlobalDomain_V3',
'Impaired_at_V3',
'LangFact_V4',
'WMF_V4',
'ProcSpeed_V4',
'MemFact_V4',
'Visuospatial_V4',
'GlobalTest_V4',
'GlobalDomain_V4',
'Impaired_at_V4',
'LangFact_V5',
'WMF_V5',
'ProcSpeed_V5',
'MemFact_V5',
'Visuospatial_V5',
'GlobalTest_V5',
'GlobalDomain_V5',
'Impaired_at_V5',
'Normed_MoCA_V3',
'Normed_MoCA_V4',
'Normed_MoCA_V2',
'stroke_time_of_day 🔒']

data = data.drop(columns=col_to_remove)

In [213]:
data.to_csv('data.csv')

In [275]:
data['Normed_MoCA_V5'].isna().sum()

np.int64(0)

In [274]:
data

,Normed_MoCA_V5
0,27.0
1,27.0
3,26.0
11,13.0
12,26.0
15,25.0
17,28.0
18,20.0
20,28.0
21,26.0


In [273]:
print(data['Normed_MoCA_V5'])
list(data.dtypes)
data = data.dropna(subset=['Normed_MoCA_V5'])

0     27.0
1     27.0
3     26.0
11    13.0
12    26.0
15    25.0
17    28.0
18    20.0
20    28.0
21    26.0
25    28.0
29    30.0
30    28.0
33    24.0
34    26.0
35    26.0
39    23.0
40    25.0
41    24.0
46    20.0
47    29.0
52    28.0
54    21.0
55    27.0
57    26.0
58    10.0
60    26.0
61    27.0
64    26.0
65    30.0
69    22.0
70    22.0
71    27.0
73    22.0
79    29.0
80    24.0
82    19.0
84    27.0
85    27.0
86    27.0
90    25.0
91    21.0
93    27.0
97    28.0
98    19.0
Name: Normed_MoCA_V5, dtype: float64


In [269]:
categorical_cols = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col].astype(str)) 

In [272]:
data

,Normed_MoCA_V5
0,27.0
1,27.0
3,26.0
11,13.0
12,26.0
15,25.0
17,28.0
18,20.0
20,28.0
21,26.0


In [255]:
print("Category Mapping:", le.classes_)

AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [267]:
X = data.drop(columns='Normed_MoCA_V5')
y = data['Normed_MoCA_V5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [265]:
#Define the space over which hyperopt will search for optimal hyperparameters.
space = {
    'max_depth': scope.int(hp.quniform("max_depth", 1, 5, 1)),
    'gamma': hp.uniform ('gamma', 0,1),
    'reg_alpha' : hp.uniform('reg_alpha', 0,50),
    'reg_lambda' : hp.uniform('reg_lambda', 10,100),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0,1),
    'min_child_weight' : hp.uniform('min_child_weight', 0,5),
    'learning_rate': hp.uniform('learning_rate', 0, .3),
    'max_bin' : scope.int(hp.quniform('max_bin', 200, 550, 1)),
    'n_estimators': 10000,
    'random_state': 42,
    'eval_metric' : 'rmse',
    'early_stopping_rounds' : 1000
    }

In [263]:
#Define the hyperopt objective.
def hyperparameter_tuning(space):
    model = XGBRegressor(**space)
    
    #Define evaluation datasets.
    evaluation = [(X_train, y_train), (X_val, y_val)]
    
    #Fit the model.
    model.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    #Obtain prediction and rmse score.
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds)
    r2 = r2_score(y_val, preds)
    mae = np.mean(np.abs(y_val - preds))
    print ("SCORE:", rmse, mae, r2)
    
    #Specify what the loss is for each model.
    return {'loss': mae, 'status': STATUS_OK, 'model': model}

In [270]:
#Run 30 trials.
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: list index out of range



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


IndexError: list index out of range

In [244]:
best_converted = {key: int(value) if key in ['max_depth', 'max_bin'] else value for key, value in best.items()}
#These fixed params dont come through via fmin so need to be applied to best 
best_converted.update({
    'eval_metric': 'rmse',
    'early_stopping_rounds': 1000,
    'n_estimators': 1000,
    'random_state': 5
})

In [245]:
bst = XGBRegressor(**best_converted)

# Train 
bst.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)], 
        verbose=False)

# predictions
preds = bst.predict(X_val)

importance = bst.feature_importances_
feature_names = X_train.columns

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

mse = mean_squared_error(y_val, preds)
r2 = r2_score(y_val, preds)
mae = np.mean(np.abs(y_val - preds))

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f}")

Mean Squared Error: 16.25
R² Score: -0.004
Mean Absolute Error: 3.50


In [246]:
bst = XGBRegressor(**best_converted)

# Train 
bst.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)], 
        verbose=False)

# predictions
preds = bst.predict(X_test)

importance = bst.feature_importances_
feature_names = X_train.columns

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = np.mean(np.abs(y_test - preds))

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f}")

Mean Squared Error: 30.56
R² Score: -0.020
Mean Absolute Error: 3.67


In [247]:
feature_importance.to_csv('feature_import.csv')

In [248]:
print(f"Train size: {X_train.shape[0]}")
print(f"Val size: {X_val.shape[0]}")  
print(f"Test size: {X_test.shape[0]}")
print(f"Total features: {X_train.shape[1]}")

Train size: 28
Val size: 8
Test size: 9
Total features: 57


In [249]:
print("Top 10 Most Important Features:")
print(feature_importance.head(10))

# Get predictions for the TEST set (not validation set)
test_preds = bst.predict(X_test)

# And see which patients were predicted
results = pd.DataFrame({
    'Row_Index': X_test.index,
    'Actual_MoCA': y_test.values,
    'Predicted_MoCA': test_preds,  # Use test predictions
    'Error': y_test.values - test_preds  # Use test predictions for error
})
print(results.head())

print("Top 10 Worst Important Features:")
print(feature_importance.tail(10))

Top 10 Most Important Features:
                         Feature  Importance
0                    Age_years 🔒         0.0
1                  stroke_AMPM 🔒         0.0
2  Left_Ventricular_Hypertrophy?         0.0
3    Atrial_Fibrillation_(Afib)?         0.0
4              Total_Cholesterol         0.0
5                            HDL         0.0
6                     Direct_LDL         0.0
7                          HbA1C         0.0
8                          crp 🔒         0.0
9     White_Cell_Count_(109/l) 🔒         0.0
   Row_Index  Actual_MoCA  Predicted_MoCA  Error
0         86         27.0            25.0    2.0
1         58         10.0            25.0  -15.0
2         60         26.0            25.0    1.0
3         97         28.0            25.0    3.0
4         80         24.0            25.0   -1.0
Top 10 Worst Important Features:
      Feature  Importance
47     IL8_V2         0.0
48     IL8_V3         0.0
49    IL18_V2         0.0
50    IL18_V3         0.0
51    MMP9_V3   

In [250]:
features_to_del = []
for i, feat in feature_importance.iterrows():
    if feat['Importance'] <= 0:
        features_to_del.append(feat['Feature'])

In [251]:
features_to_del

['Age_years 🔒',
 'stroke_AMPM 🔒',
 'Left_Ventricular_Hypertrophy?',
 'Atrial_Fibrillation_(Afib)?',
 'Total_Cholesterol',
 'HDL',
 'Direct_LDL',
 'HbA1C',
 'crp 🔒',
 'White_Cell_Count_(109/l) 🔒',
 'Neutrophils_(109/l) 🔒',
 'Eosinophils_(109/l) 🔒',
 'Platelet_Count_(109/l) 🔒',
 'Previous Stroke 🔒',
 'Hypertension 🔒',
 'Hyperlipidemia 🔒',
 'Smoking_Status 🔒',
 'Alcohol_Consumed 🔒',
 'Frequency 🔒',
 'no_vasc_comorbs 🔒',
 'Index of Multiple Deprivation Decile',
 'Income Score',
 'Employment Decile',
 'Employment Score',
 'Education and Skills Decile',
 'Health and Disability Decile',
 'Crime Decile',
 'Barriers to Housing and Services Decile',
 'IDACI Rank',
 'IDAOPI Rank',
 'CCL2_V2',
 'CCL2_V5',
 'CCL3_V2',
 'CCL3_V3',
 'CD163_V2',
 'CD163_V3',
 'CRP_V2',
 'CRP_V3',
 'CRP_V4',
 'gp130_V2',
 'gp130_V3',
 'IL1Ra_V2',
 'IL1Ra_V3',
 'IL1Ra_V4',
 'IL6_V2',
 'IL6_V3',
 'IL6_V4',
 'IL8_V2',
 'IL8_V3',
 'IL18_V2',
 'IL18_V3',
 'MMP9_V3',
 'MMP9_V4',
 'PDGFBB_V3',
 'PDGFBB_V4',
 'sIL6R_V3',
 'sIL

In [252]:
data = data.drop(columns=features_to_del)